# Payload Logging in KFServing

Deploy message-dumper which will persist the logs collected from the inferenceservice's requests and responses.

In [ ]:
import textwrap
dumper_yaml = textwrap.dedent("""\
apiVersion: serving.knative.dev/v1alpha1
kind: Service
metadata:
  name: message-dumper
spec:
  template:
    metadata:
        annotations:
          autoscaling.knative.dev/minScale: "1"
    spec:
      containers:
      - image: gcr.io/knative-releases/knative.dev/eventing-contrib/cmd/event_display
""")
f = open("dumper.yaml", "w")
f.write(dumper_yaml)
f.close()

In [ ]:
!kubectl apply -f dumper.yaml

Start the sklearn-iris inferenceservice

In [ ]:
import textwrap
sklearn_yaml = textwrap.dedent("""\
apiVersion: "serving.kubeflow.org/v1alpha2"
kind: "InferenceService"
metadata:
  name: "sklearn-iris"
spec:
  default:
    predictor:
      logger:
          url: http://message-dumper.default/
          mode: all
      sklearn:
        storageUri: "gs://kfserving-samples/models/sklearn/iris"
""")
f = open("sklearn.yaml", "w")
f.write(sklearn_yaml)
f.close()

In [ ]:
!kubectl apply -f sklearn.yaml

Check that the sklearn server and message-dumper are ready.

In [ ]:
!kubectl get pod | grep message-dumper
!kubectl get pod | grep sklearn-iris
!kubectl get inferenceservice sklearn-iris

Once the model and message-dumper are ready, we then can do model a prediction using a simple curl command.

In [ ]:
%%bash
MODEL_NAME=sklearn-iris
SERVICE_HOSTNAME=$(kubectl get inferenceservice sklearn-iris -o jsonpath='{.status.url}' | cut -d "/" -f 3)
curl -v -H "Host: ${SERVICE_HOSTNAME}" http://$kubeflow_url/v1/models/$MODEL_NAME:predict -d '{"instances": [[6.8,  2.8,  4.8,  1.4],[6.0,  3.4,  4.5,  1.6]]}'

Check for cloud-event logs in the message-dumper

In [ ]:
res = !kubectl logs $(kubectl get pod -l serving.knative.dev/service=message-dumper -o jsonpath='{.items[0].metadata.name}') user-container

Parse through the cloud-events logs to collect all of the requests and responses.

In [ ]:
def parse_events(stdout):
    line_split = stdout.split('\n')
    line_iter = iter(line_split)

    depth_stack = [] # A stack to describe what is currently being parsed
    item_stack = []
    cur_item = {"key": "Data", "value": {}}

    events = []

    while True:
        try:
            line = next(line_iter).strip()
            if 'Data,' in line:
                depth_stack.append(ParseItem.DATA)
                item_stack.append(cur_item)
            elif '{' in line:
                depth_stack.append(ParseItem.OBJECT)
                item_stack.append(cur_item)
                if "\"" in line:
                    key = line.split('\"')[1]
                    cur_item = {"key": key, "value": {}}
                else:
                    cur_item = {"value": {}}
            elif '}' in line:
                if depth_stack.pop() != ParseItem.OBJECT:
                    print("Parsing failed: invalid string")
                    sys.exit()
                next_item = item_stack.pop()
                if "key" in cur_item.keys():
                    next_item["value"][cur_item["key"]] = cur_item["value"]
                    cur_item = next_item
                else:
                    try:
                        next_item["value"].append(cur_item["value"])
                        cur_item = next_item
                    except: # Root case
                        events.append(cur_item["value"])

                        depth_stack = []
                        item_stack = []
                        cur_item = {"key": "Data", "value": {}}
            elif '[' in line:
                depth_stack.append(ParseItem.LIST)
                item_stack.append(cur_item)
                if "\"" in line:
                    key = line.split('\"')[1]
                    cur_item = {"key": key, "value": []}
                else:
                    cur_item = {"value": []}
            elif ']' in line:
                #print_info(events, depth_stack, item_stack, cur_item)
                if depth_stack.pop() != ParseItem.LIST:
                    print("Parsing failed: invalid string")
                    sys.exit()
                next_item = item_stack.pop()
                if "key" in cur_item.keys():
                    next_item["value"][cur_item["key"]] = cur_item["value"]
                    cur_item = next_item
                else:
                    try:
                        next_item["value"].append(cur_item["value"])
                        cur_item = next_item
                    except:  # Root case
                        events.append(cur_item["value"])

                        depth_stack = []
                        item_stack = []
                        cur_item = {"key": "Data", "value": {}}
            elif len(depth_stack) > 0:
                try:
                    cur_item["value"].append(line.replace(',', ''))
                except:
                    key = line.split('\"')[1]
                    value = line[line.index(":") + 1: len(line) - 1]
                    cur_item["value"][key] = value
        except:
            for x in range(0, len(events)):
                print("Event (", x, "):", events[x])
            return events
events = parse_events(res)

In [ ]:
## clean up the previous examples
!kubectl delete -f sklearn.yaml
!kubectl delete -f dumper.yaml